In [37]:
import requests
import os
from dotenv import load_dotenv
import json
import pandas as pd

In [38]:
load_dotenv()

True

In [39]:
customer_id = os.getenv("customer_id")
customer_secret = os.getenv("customer_secret")
refresh_token = os.getenv("refresh_token")
print(refresh_token)

e03082925525ef032c02855c5caea26a120429f8


In [40]:
if not all([customer_id, customer_secret, refresh_token]):
    raise ValueError("Missing Strava API Credentials from .env file")

In [41]:
token_url = "https://www.strava.com/oauth/token"
activities_url = "https://www.strava.com/api/v3/athlete/activities"

In [42]:
# def refresh_access_token():
#     """Refreshes the access token using the refresh token."""
#     payload = {
#         "client_id": customer_id,
#         "client_secret": customer_secret,
#         "grant_type": "refresh_token",
#         "refresh_token": refresh_token,
#     }

#     try:
#         response = requests.post(token_url, data=payload)
#         response.raise_for_status()
#         token_data = response.json()
#         access_token = token_data["access_token"]
#         return access_token
#     except requests.exceptions.RequestException as e:
#         print(f"Error refreshing access token: {e}")
#         return None 


In [43]:
def refresh_access_token():
    """Refreshes the access token and prints scope information."""
    payload = {
        "client_id": customer_id,
        "client_secret": customer_secret,
        "grant_type": "refresh_token",
        "refresh_token": refresh_token,
    }

    try:
        response = requests.post(token_url, data=payload)
        response.raise_for_status()

        token_data = response.json()
        print(json.dumps(token_data, indent=4))  # Print the FULL response for debugging

        access_token = token_data.get("access_token")  # Safer dictionary access
        scopes = token_data.get("scope") # Get the scope. It can be a string or a list
        print(f"Access Token Scopes: {scopes}")  # Print the scopes

        return access_token

    except requests.exceptions.RequestException as e:
        print(f"Error refreshing access token: {e}")
        try:
            error_data = response.json()
            print(f"Strava API Error Details: {json.dumps(error_data, indent=4)}")
        except json.JSONDecodeError:
            print(f"Error response content: {response.text}")
        return None

In [44]:
# def get_strava_activities(access_token, per_page=30):
#     """Retrieves Strava activities."""
#     headers = {"Authorization": f"Bearer {access_token}"}
#     params = {"per_page": per_page}

#     try:
#         response = requests.get(activities_url, headers=headers, params=params)
#         response.raise_for_status()
#         activities = response.json()
#         return activities
#     except requests.exceptions.RequestException as e:
#         print(f"Error getting activities: {e}")
#         return None

# if __name__ == "__main__":
#     access_token = refresh_access_token()

#     if access_token:
#         activities = get_strava_activities(access_token)

#         if activities:
#             print("Successfully retrieved activities!")
#             #print(json.dumps(activities, indent=4))  # Pretty print JSON for inspection

#             for activity in activities:
#                 print(activity.get("name", "Unknown Activity"))

#             # Now you have the activities list; proceed with your analysis.

#         else:
#             print("Failed to retrieve activities.")
#     else:
#         print("Failed to refresh access token.")

In [45]:
def get_strava_activities(access_token, per_page=30):
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"per_page": per_page}

    try:
        response = requests.get(activities_url, headers=headers, params=params)
        response.raise_for_status()  # Check for HTTP errors

        activities = response.json()

        if activities:  # Check if the activities list is not empty
            return activities
        else:
            print("No activities found.") # Explicitly print if no activities
            return []  # Return an empty list

    except requests.exceptions.RequestException as e:
        print(f"Error getting activities: {e}")
        if response.status_code != 200: # Check for non-200 status codes
            try:
                error_data = response.json()  # Attempt to parse JSON error response
                print(f"Strava API Error Details: {json.dumps(error_data, indent=4)}") # Print the error details
            except json.JSONDecodeError:
                print(f"Error response content: {response.text}") # Print raw text if not JSON
        return None

if __name__ == "__main__":
    access_token = refresh_access_token()

    if access_token:
        activities = get_strava_activities(access_token)

        if activities is not None:  # Check for None (API error)
            if activities: # Check if list is not empty
                print("Successfully retrieved activities!")
                #print(json.dumps(activities, indent=4))  # Pretty print JSON for inspection

                for activity in activities:
                    print(activity.get("name", "Unknown Activity"))
            else:
                print("No activities returned.") # Explicitly state no activities
        else:
            print("Failed to retrieve activities due to API error.") # Indicate API error
    else:
        print("Failed to refresh access token.")


{
    "token_type": "Bearer",
    "access_token": "59b6f758be1c8874f3f29d553baf81c690eebc07",
    "expires_at": 1740366176,
    "expires_in": 20453,
    "refresh_token": "e03082925525ef032c02855c5caea26a120429f8"
}
Access Token Scopes: None
Error getting activities: 401 Client Error: Unauthorized for url: https://www.strava.com/api/v3/athlete/activities?per_page=30
Strava API Error Details: {
    "message": "Authorization Error",
    "errors": [
        {
            "resource": "AccessToken",
            "field": "activity:read_permission",
            "code": "missing"
        }
    ]
}
Failed to retrieve activities due to API error.


In [46]:
authorization_url = f"https://www.strava.com/oauth/authorize?client_id={101544}&redirect_uri=http://localhost:8000/callback&response_type=code&scope=activity:read"
print(f"Authorization URL: {authorization_url}")

Authorization URL: https://www.strava.com/oauth/authorize?client_id=101544&redirect_uri=http://localhost:8000/callback&response_type=code&scope=activity:read


In [ ]:
https://www.strava.com/oauth/authorize?client_id=<your_client_id>&redirect_uri=http://localhost:8000&response_type=code&scope=activity:read